## APRENDIZADO DE MÁQUINA NÃO SUPERVISIONADO:

## Redução de dimensionalidade utilizando o algoritmo PCA (Principal Componentes Analysis)

O PCA é um dos principais algoritmos de aprendizagem de máquina não supervisionada.

Ele identifica a correlação entre variáveis e, caso haja forte correlação entre elas, decide que é possível reduzir a dimensionalidade dos dados.

Supondo um número m de variáveis, o PCA extrai um novo número p <= m que explica a melhor variação na base de dados, excluindo a variável dependente y. Tal número m é escolhido pelo projetista.


# Parte 1: Importação de bibliotecas

In [1]:
import pandas as pd

# Conversão de dados categóricos em numéricos:
from sklearn.preprocessing import LabelEncoder
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
# Sugestão de leitura: Como fazer LabelEncoder em um DataFrame — Python/SKLearn
# https://gianmedeirao.medium.com/como-fazer-labelencoder-em-um-dataframe-python-sklearn-655ba2c6ae7e

# Normalização dos dados
from sklearn.preprocessing import StandardScaler
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
# Sugestão de leitura: How to Use StandardScaler and MinMaxScaler Transforms in Python:
# https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/
# https://www.hashtagtreinamentos.com/padronizacao-e-normalizacao-em-ciencia-de-dados

from sklearn.model_selection import train_test_split
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
# Sugestão de leitura:
# https://www.geeksforgeeks.org/how-to-do-train-test-split-using-sklearn-in-python/

from sklearn.decomposition import PCA
# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

# Classificador Random Forest
from sklearn.ensemble import RandomForestClassifier
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

# Pacote para cálculo da acurácia do modelo de classificação:
from sklearn.metrics import accuracy_score
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

# Parte 2: Preparação dos dados

O conjunto de dados modificado "census.csv" consiste em aproximadamente 32.000 registros de dados, com cada registro de dados tendo 13 características.

https://archive.ics.uci.edu/dataset/20/census+income

Este conjunto de dados é uma versão modificada do conjunto de dados publicado no artigo "*Scaling Up the Accuracy of Naive-Bayes Classifiers: a Decision-Tree Hybrid*", de Ron Kohavi.

Artigo:

https://aaai.org/papers/033-scaling-up-the-accuracy-of-naive-bayes-classifiers-a-decision-tree-hybrid/


In [2]:
# Criando um dataframe a partir do dados do arquivo
df = pd.read_csv('/content/census.csv')
df.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
df.shape

(32561, 15)

In [4]:
# Seleção de atributos (Dados de entrada)
# Com o método .iloc selecionamos (por índice) todas as linhas,
# e as colunas de 0 a 13 (o segundo parâmetro é excludente)
X = df.iloc[:, 0:14].values

In [5]:
X

array([[39, ' State-gov', 77516, ..., 0, 40, ' United-States'],
       [50, ' Self-emp-not-inc', 83311, ..., 0, 13, ' United-States'],
       [38, ' Private', 215646, ..., 0, 40, ' United-States'],
       ...,
       [58, ' Private', 151910, ..., 0, 40, ' United-States'],
       [22, ' Private', 201490, ..., 0, 20, ' United-States'],
       [52, ' Self-emp-inc', 287927, ..., 0, 40, ' United-States']],
      dtype=object)

In [6]:
# Seleção do target (Dados de saída)
# Com o método .iloc selecionamos (por índice) todas as linhas, apenas a coluna 14 (income)
y = df.iloc[:, 14].values
# y = df['income'].values

In [7]:
y

array([' <=50K', ' <=50K', ' <=50K', ..., ' <=50K', ' <=50K', ' >50K'],
      dtype=object)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   final-weight    32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loos    32561 non-null  int64 
 12  hour-per-week   32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [9]:
# Em Machine Learning, o "Label Encoding" é o processo de conversão
# de variáveis categóricas em variáveis numéricas (inteiros).

# Em nosso dataframe, as colunas com variáveis categóricas (e seus índices) são:
# workclass (1), education (3), marital (5), occupation (6), relationship (7), race (8), sex (9) e country (13)

# Criaremos variáveis com o nome das colunas e aplicaremos a transformação para cada uma deles com o LabelEncoder():
le_workclass = LabelEncoder()
le_education = LabelEncoder()
le_marital = LabelEncoder()
le_occupation = LabelEncoder()
le_relationship = LabelEncoder()
le_race = LabelEncoder()
le_sex = LabelEncoder()
le_country = LabelEncoder()

In [10]:
# Usando o método .fit_transform aplicamos os novos valores transformados às colunas:
X[:,1] = le_workclass.fit_transform(X[:,1])
X[:,3] = le_education.fit_transform(X[:,3])
X[:,5] = le_marital.fit_transform(X[:,5])
X[:,6] = le_occupation.fit_transform(X[:,6])
X[:,7] = le_relationship.fit_transform(X[:,7])
X[:,8] = le_race.fit_transform(X[:,8])
X[:,9] = le_sex.fit_transform(X[:,9])
X[:,13] = le_country.fit_transform(X[:,13])

In [11]:
X

array([[39, 7, 77516, ..., 0, 40, 39],
       [50, 6, 83311, ..., 0, 13, 39],
       [38, 4, 215646, ..., 0, 40, 39],
       ...,
       [58, 4, 151910, ..., 0, 40, 39],
       [22, 4, 201490, ..., 0, 20, 39],
       [52, 5, 287927, ..., 0, 40, 39]], dtype=object)

In [12]:
# Padronização
# É o redimensionamento dos recursos para criar um “padrão” garantindo
# que os novos dados tenham média zero e desvio padrão igual a 1
scaler = StandardScaler()

# Salvando os novos dados já padronizados:
X = scaler.fit_transform(X)

In [13]:
X

array([[ 0.03067056,  2.15057856, -1.06361075, ..., -0.21665953,
        -0.03542945,  0.29156857],
       [ 0.83710898,  1.46373585, -1.008707  , ..., -0.21665953,
        -2.22215312,  0.29156857],
       [-0.04264203,  0.09005041,  0.2450785 , ..., -0.21665953,
        -0.03542945,  0.29156857],
       ...,
       [ 1.42360965,  0.09005041, -0.35877741, ..., -0.21665953,
        -0.03542945,  0.29156857],
       [-1.21564337,  0.09005041,  0.11095988, ..., -0.21665953,
        -1.65522476,  0.29156857],
       [ 0.98373415,  0.77689313,  0.92989258, ..., -0.21665953,
        -0.03542945,  0.29156857]])

In [14]:
# Com o método train_test_split separamos os dados em treino e teste:
# X_treino e y_treino: dados para treinamento dos modelos (80% dos dados iniciais)
# X_teste e y_teste: dados para teste e avaliação dos modelos (20% dos dados iniciais)

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.20, random_state=10)

In [15]:
X_treino.shape, X_teste.shape

((26048, 14), (6513, 14))

## Parte 3: Aplicação do algoritmo PCA (Principal component analysis)

In [22]:
# Criamos a variável pca e aplicamos o algoritmo PCA para a redução do número original de colunas para
# um número definido "p"
# O hiperparâmetro "n_components" recebe o valor definido de novas colunas (p)
p = 3
pca = PCA(n_components=p)

In [23]:
# Definimos novas variáveis de treino e testes (agora com um menor números de colunas)
X_treino_pca = pca.fit_transform(X_treino)
X_teste_pca = pca.transform(X_teste)

In [18]:
X_treino.shape, X_teste.shape

((26048, 14), (6513, 14))

In [24]:
X_treino_pca.shape, X_teste_pca.shape

((26048, 3), (6513, 3))

In [25]:
# O método "explained_variance_ratio" permite calcular a
# porcentagem de variância dos dados que consegue ser explicada por cada um dos componentes selecionados

# Relembrando! :-)
# A variância, assim como o desvio padrão, é uma das medidas de dispersão de dados
# que mostra o comportamento dos dados de uma amostra em relação a uma medida central (por exemplo, a média)

pca.explained_variance_ratio_

array([0.15127989, 0.10026403, 0.08964636])

In [26]:
# Somando os percentuais conseguimos ver o quanto nosso
# modelo PCA conseguiu explicar dos dados de entrada
total_variancia = pca.explained_variance_ratio_.sum()
print(f'O modelo PCA com {p} variáveis explica{100*(total_variancia): .2f} % dos dados de entrada')

O modelo PCA com 3 variáveis explica 34.12 % dos dados de entrada


# Parte 4: Aplicando os novos dados gerados em um modelo de classificação.

Utilizaremos o algoritmo de classificação Random Forest com base nos novos dados, após a aplicação da técnica de redução de dimensionalidade.

In [27]:
# Instanciando o objeto do classificador
modelo_rf = RandomForestClassifier(n_estimators=40, random_state=0)

In [33]:
# Usando o método .fit para treinar o modelo

# Com base nos dados de treino iniciais:
# modelo_rf.fit(X_treino, y_treino)

# Com base nos dados de treino gerados após a aplicação do algoritmo PCA:
modelo_rf.fit(X_treino_pca, y_treino)

RandomForestClassifier(n_estimators=40, random_state=0)

In [34]:
# Usando o método .predict para estimar os resultados

# Realizando as previsões com base nos dados de teste iniciais
# previsoes = modelo_rf.predict(X_teste)

# Realizando as previsões com base nos dados de teste após a aplicação do algoritmo PCA
previsoes = modelo_rf.predict(X_teste_pca)


In [30]:
previsoes

array([' <=50K', ' <=50K', ' >50K', ..., ' >50K', ' <=50K', ' <=50K'],
      dtype=object)

In [31]:
y_teste

array([' <=50K', ' <=50K', ' >50K', ..., ' >50K', ' <=50K', ' <=50K'],
      dtype=object)

In [35]:
# Calculando a acurácia do modelo de classificação:
acuracia = accuracy_score(y_teste, previsoes)
print(f'A acurácia do modelo Random Forest é de:{(100*acuracia): .2f} %')

# Sugestão de leitura:
# Métricas de avaliação de modelos de aprendizado de máquina:
# https://mariofilho.com/as-metricas-mais-populares-para-avaliar-modelos-de-machine-learning/

A acurácia do modelo Random Forest é de: 79.07 %


# Parte 5: Sugestões de exercícios

- Volte nas células da Parte 3 e altere os valores de p.

- Recalcule a média de variância dos dados após a aplicação do algoritmo PCA.

- Treine novamente o modelo de classificação, realize as predições e compare os resultados.

- Utilize um novo dataset e repita os passos.